In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from category_encoders import  LeaveOneOutEncoder, BinaryEncoder, TargetEncoder
import time
import logging

sample_submission = pd.read_csv("../input/cat-in-the-dat-ii/sample_submission.csv")
test = pd.read_csv("../input/cat-in-the-dat-ii/test.csv")
train = pd.read_csv("../input/cat-in-the-dat-ii/train.csv")

In [2]:
logging.basicConfig(level=logging.INFO, filename='log_estimators.log')

In [3]:
def replace_nans(dataframe):
    for each in dataframe.columns:
        if each == 'id':
            continue
        dataframe.loc[:, each] = dataframe.fillna(dataframe[each].mode()[0])
    
    return dataframe

In [4]:
def encoder(dataframe, columns, enc_type='bin'):

	if enc_type == 'bin':
		for col in columns:
			unique = dataframe[col].unique()
			dataframe.loc[:, col] = \
			dataframe[col].apply(lambda x: 1 if x==unique[0] else (0 if x==unique[1] else None))
	if enc_type == 'ord':
		encoder = OrdinalEncoder(dtype=np.int16)
		for col in columns:
			dataframe.loc[:, col] = encoder.fit_transform(np.array(dataframe[col]).reshape(-1,1))


	return dataframe

In [5]:
def rank_features(estimator, X_train, y_train):
	selector = RFE(estimator, 10, step=1)
	selector = selector.fit(X_train, y_train)
	return selector.ranking_

In [6]:
def fitter(clf, X_train, X_test, y_train, y_test):
    print('training ', clf)
    y_train = np.array([[target] for target in y_train])
    y_test = np.array([[target] for target in y_test])
    clf.fit(X_train, y_train)
    #predictions = clf.predict(X_test)
    #print('accuracy:', accuracy_score(y_test, predictions))
    print('cross_val_score:', cross_val_score(clf, X_train, y_train))
    
    return clf

In [7]:
def main_2():
    data = train
    data = replace_nans(data)
    submission_data = replace_nans(test)
    print(data.columns)
    nom_cols = ['nom_0', 'nom_1', 'nom_2']
    ord_cols = ['ord_3', 'ord_4', 'ord_5']
    bin_cols = ['bin_3', 'bin_4']
    
    ord_encoder = OrdinalEncoder()
    for enc in ord_cols+nom_cols:
        data[enc] = ord_encoder.fit_transform(np.array(data[enc]).reshape(-1,1))
        submission_data[enc] = ord_encoder.fit_transform(np.array(submission_data[enc]).reshape(-1,1))
    
    
    for enc in ['nom_3','nom_4']:
        enc1 = pd.get_dummies(data[enc], prefix=enc)
        data.drop(columns=enc, inplace=True)
        data = pd.concat([data, enc1], axis=1)
        
    for enc in ['nom_3','nom_4']:
        enc1 = pd.get_dummies(submission_data[enc], prefix=enc)
        submission_data.drop(columns=enc, inplace=True)
        submission_data = pd.concat([submission_data, enc1], axis=1)
           
    
    target = data['target']
    data = data.drop('target', axis=1)
    #for enc in ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']:
    loo_enc = LeaveOneOutEncoder(cols=['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'], return_df=True)
    loo_enc.fit(data, target)
    data = loo_enc.transform(data)
    #print(data)
    #submission_data[enc] = target_enc.transform(submission_data[enc].values.reshape(-1, 1))
    submission_data = loo_enc.transform(submission_data)
    
    data = encoder(data, ['ord_1', 'ord_2'], enc_type='ord')
    data = encoder(data, bin_cols, enc_type='bin')
    submission_data = encoder(submission_data, ['ord_1', 'ord_2'], enc_type='ord')
    submission_data = encoder(submission_data, bin_cols, enc_type='bin')
    time_features = ['day', 'month']
    
    for feature in time_features:
        data[feature+'_sin'] = np.sin((2*np.pi*data[feature])/max(data[feature]))
        data[feature+'_cos'] = np.cos((2*np.pi*data[feature])/max(data[feature]))

    data.drop(time_features, axis=1, inplace=True)
    
    for feature in time_features:
        submission_data[feature+'_sin'] = np.sin((2*np.pi*submission_data[feature])/max(submission_data[feature]))
        submission_data[feature+'_cos'] = np.cos((2*np.pi*submission_data[feature])/max(submission_data[feature]))

    submission_data.drop(time_features, axis=1, inplace=True)

    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25)
    clf_2 = LogisticRegression(solver='saga', verbose=1)
    #clf_3 = GradientBoostingClassifier(n_estimators=500, verbose=1)
    #clf_4 = PassiveAggressiveClassifier(verbose=1)
    clf_5 = RandomForestClassifier(n_estimators=500, verbose=2) #, criterion='entropy')
    
    clf = fitter(clf_5, X_train, X_test, y_train, y_test)
    predictions = clf.predict_proba(submission_data.values)
    predictions = [x[1] for x in predictions]
    #print(predictions)
    submission_data = pd.read_csv("../input/cat-in-the-dat-ii/test.csv")
    submission_data['target'] = predictions
    submission_data = pd.concat([submission_data['id'], submission_data['target']], axis=1)
    submission_data.to_csv('submission.csv', index=False)

In [8]:
main_2()

Index(['id', 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
       'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month',
       'target'],
      dtype='object')
training  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=2, warm_start=False)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 500


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  9.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   31.8s finished


cross_val_score: [0.81922787 0.81868667 0.81842546]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.4min finished
